<a href="https://colab.research.google.com/github/ajnovice/SecureAI/blob/master/Federated_Learning_Project_MNSIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install syft


In [21]:
!pip install numpy

In [0]:
import torch as th
from torch import nn, optim

In [23]:
import syft as sy  
hook = sy.TorchHook(th) 
bob = sy.VirtualWorker(hook, id="bob")  
alice = sy.VirtualWorker(hook, id="alice")
jon = sy.VirtualWorker(hook,id="jon")

W0714 22:40:28.290601 139847980472192 hook.py:98] Torch was already hooked... skipping hooking process


In [24]:
bob.add_workers([alice,jon])
alice.add_workers([alice,bob])
jon.add_workers([alice,bob])

W0714 22:40:28.310251 139847980472192 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0714 22:40:28.313383 139847980472192 base.py:628] Worker jon already exists. Replacing old worker which could cause                     unexpected behavior
W0714 22:40:28.315045 139847980472192 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0714 22:40:28.316157 139847980472192 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0714 22:40:28.317377 139847980472192 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0714 22:40:28.318498 139847980472192 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:jon #objects:0>

In [25]:
model = nn.Linear(2,1)
th.__version__
model.weight

Parameter containing:
tensor([[ 0.6556, -0.5956]], requires_grad=True)

In [0]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [29]:
bob._objects

{10828868189: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 20342157159: tensor([[1.],
         [1.]], requires_grad=True)}

In [30]:
alice._objects

{19443924385: tensor([[1., 0.],
         [0., 0.]], requires_grad=True), 23559274091: tensor([[0.],
         [0.]], requires_grad=True)}

In [31]:
jon._objects

{}

In [0]:
def train():
  for i in range(10):
    bob_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)
    bob_optim=optim.SGD(params=bob_model.parameters(), lr=0.1)
    alice_optim=optim.SGD(params=alice_model.parameters(), lr=0.1)
    for k in range(10):
      bob_optim.zero_grad()
      boutput = bob_model(data_bob)
      bloss =((target_bob-boutput)**2).sum()
      bloss.backward()
      bob_optim.step()
      
      alice_optim.zero_grad()
      aoutput = alice_model(data_alice)
      aloss = ((target_alice-aoutput)**2).sum()
      alice_optim.step()
    bob_model.move(jon)
    alice_model.move(jon)
    model.weight.data.set_(((alice_model.weight.data + bob_model.weight.data)/2).get())
    model.bias.data.set_(((alice_model.bias.data + bob_model.bias.data)/2).get())
    print("A loss : ",aloss.item())
    print("B loss : ",bloss.item())
      

In [49]:
train()

AttributeError: ignored

In [34]:
model.w


Parameter containing:
tensor([[ 0.6556, -0.5956]], requires_grad=True)